In [7]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib

# Añadir src/ al path para poder importar config
PROJECT_ROOT = pathlib.Path().resolve().parent.parent  # estamos en /notebooks/lstm5d
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg
from sklearn.preprocessing import StandardScaler   

In [8]:
# Leer precios desde parquet
df = pd.read_parquet(cfg.DATA / "raw" / "prices.parquet")
df = df.ffill().dropna()

# Cargar VIX y alinearlo con los precios
vix = pd.read_parquet(cfg.DATA / "raw" / "vix.parquet").squeeze()
vix = vix.ffill().reindex(df.index).ffill()
vix_t1 = vix.shift(1).rename('vix_t-1')

# Calcular retornos logarítmicos
ret = np.log(df / df.shift(1)).dropna()
ret.head()

Ticker,AAPL,ABT,ADBE,AMZN,BAC,BTC-USD,COST,CRM,CSCO,CVX,...,PFE,PG,T,TSLA,UNH,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-21,0.056626,0.003890,0.022111,0.019725,-0.027439,0.0,0.009480,0.023556,0.012071,0.012415,...,-0.001330,-0.002049,-0.000892,0.042968,0.028485,0.031288,-0.004585,0.014758,0.009724,0.006972
2012-05-22,-0.007708,0.002908,0.000000,-0.012828,0.021725,0.0,-0.004189,0.002680,0.003593,-0.003719,...,-0.007571,-0.003793,-0.002680,0.068181,0.003055,0.018500,0.001209,0.008562,0.010886,-0.001097
2012-05-23,0.024107,-0.010052,0.005297,0.009015,0.026856,0.0,0.002464,0.007732,-0.002394,-0.003227,...,-0.012596,-0.012108,-0.005381,0.007118,-0.006482,0.006567,-0.002661,0.002208,0.013249,0.001097
2012-05-24,-0.009226,0.016965,-0.020089,-0.009433,-0.004193,0.0,0.013947,-0.029650,-0.018138,0.010650,...,0.002261,0.002881,0.008358,-0.024145,0.015415,0.005022,0.002661,0.002203,0.007559,0.006924
2012-05-25,-0.005374,-0.002728,0.001901,-0.010978,0.001400,0.0,0.000000,0.005389,-0.003667,-0.012065,...,-0.000452,-0.001279,0.001485,-0.015644,-0.001780,-0.003346,0.001449,0.001570,0.003682,-0.006436


In [9]:
# 🔧 CAMBIO CRÍTICO: TARGET DIARIO (como LSTM-1d)
print("🔧 CAMBIO: Target de retorno acumulado 5d → retorno diario")

# Retorno acumulado 5 días (solo para momentum)
ret5 = ret.rolling(5).sum()

# Volatilidad 5 días (solo para momentum)
vol5 = ret.rolling(5).std()

# Momentum = retorno / volatilidad
momentum = (ret5 / (vol5 + 1e-6)).clip(-10, 10)

# 🎯 TARGET DIARIO: retorno 1 día futuro (como LSTM-1d) 
# ✅ CRITICAL FIX: Asegurar que NO HAY LOOK-AHEAD BIAS
y = ret.shift(-1).dropna()  # Predecir retorno t+1 usando info hasta t-1

# Alinear features al target
ret = ret.shift(1).reindex(y.index)
momentum = momentum.shift(1).reindex(y.index)
vix_t1 = vix_t1.reindex(y.index)

print(f"✅ Nuevo target: retorno diario")
print(f"   Target shape: {y.shape}")
print(f"   Target range: [{y.min().min():.4f}, {y.max().max():.4f}]")
print(f"   Target mean: {y.mean().mean():.6f}")

# Visual check
ret.tail(3), momentum.tail(3), y[-3:]


🔧 CAMBIO: Target de retorno acumulado 5d → retorno diario
✅ Nuevo target: retorno diario
   Target shape: (4519, 40)
   Target range: [-0.5507, 6.1254]
   Target mean: 0.000533


(Ticker          AAPL       ABT      ADBE      AMZN       BAC   BTC-USD  \
 Date                                                                     
 2025-06-23  0.000000  0.000000  0.000000  0.000000  0.000000 -0.012500   
 2025-06-24  0.002484  0.002028  0.008454 -0.005835  0.017430  0.044455   
 2025-06-25 -0.005973  0.035531  0.005823  0.020417  0.007745  0.004422   
 
 Ticker          COST       CRM      CSCO       CVX  ...       PFE        PG  \
 Date                                                ...                       
 2025-06-23  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
 2025-06-24  0.024377  0.010610  0.015857 -0.018151  ...  0.002500  0.012183   
 2025-06-25 -0.002552  0.029034  0.007541 -0.022796  ...  0.011585 -0.004169   
 
 Ticker             T      TSLA       UNH         V        VZ       WFC  \
 Date                                                                     
 2025-06-23  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000

In [11]:
# ╔════════════════════════════════════════════════════════════════╗
# REGENERAR FEATURES NECESARIAS (para evitar NameError)
# ╚════════════════════════════════════════════════════════════════╝
print("🔧 Regenerando features necesarias...")

# Regenerar retornos multi-horizonte
ret_1d = ret.copy()
ret_3d = ret.rolling(3).sum()
ret_5d = ret.rolling(5).sum()
ret_10d = ret.rolling(10).sum()

# Regenerar volatilidades jerárquicas
vol_5d = ret.rolling(5).std()
vol_20d = ret.rolling(20).std()
vol_60d = ret.rolling(60).std()

# Regenerar momentum temporal
mom_3_20 = (ret_3d / (vol_5d + 1e-6)).clip(-10, 10)
mom_5_60 = (ret_5d / (vol_20d + 1e-6)).clip(-10, 10)

# Regenerar VIX features avanzadas
vix_level = (vix / 100)
vix_change_1d = vix.pct_change(1).clip(-1, 1)
vix_change_5d = vix.pct_change(5).clip(-2, 2)
vix_trend = (vix.rolling(5).mean() / vix.rolling(20).mean()).clip(0.5, 2.0)

# Regenerar correlaciones cross-asset (FIX corrwith issue)
market_ret = ret.mean(axis=1)
market_corr = pd.DataFrame(index=ret.index, columns=ret.columns)
for col in ret.columns:
    market_corr[col] = ret[col].rolling(20).corr(market_ret).clip(-1, 1)

# Regenerar ratios técnicos
ret_ratio_3_10 = (ret_3d / (ret_10d.abs() + 1e-6)).clip(-5, 5)
vol_ratio_5_20 = (vol_5d / (vol_20d + 1e-6)).clip(0, 10)

# Regenerar target
y = ret.shift(-1).dropna()

print("✅ Features regeneradas correctamente")
print(f"   ret_ratio_3_10 shape: {ret_ratio_3_10.shape}")
print(f"   vol_ratio_5_20 shape: {vol_ratio_5_20.shape}")
print(f"   market_corr shape: {market_corr.shape}")
print(f"   y shape: {y.shape}")


🔧 Regenerando features necesarias...
✅ Features regeneradas correctamente
   ret_ratio_3_10 shape: (4519, 40)
   vol_ratio_5_20 shape: (4519, 40)
   market_corr shape: (4519, 40)
   y shape: (4518, 40)


In [12]:
# ╔════════════════════════════════════════════════════════════════╗
# CONSTRUCCIÓN DE SECUENCIAS CON FEATURES AVANZADAS (484 features)
# ╚════════════════════════════════════════════════════════════════╝

# ── ALINEAR TODAS LAS FEATURES AL TARGET ────────────────────────
print("🔧 Alineando features avanzadas al target...")

# ✅ CRITICAL: Features hasta t-1, target en t+1 (NO LOOK-AHEAD)
# Alinear features t-1 al índice del target  
ret_1d = ret_1d.shift(2).reindex(y.index)  # ✅ shift(2): usa ret hasta t-2 para predecir t+1
ret_3d = ret_3d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
ret_5d_feat = ret_5d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
ret_10d = ret_10d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vol_5d = vol_5d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vol_20d = vol_20d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia 
vol_60d = vol_60d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
mom_3_20 = mom_3_20.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
mom_5_60 = mom_5_60.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
ret_ratio_3_10 = ret_ratio_3_10.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vol_ratio_5_20 = vol_ratio_5_20.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
market_corr = market_corr.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia

# VIX features alineadas
vix_level = vix_level.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vix_change_1d = vix_change_1d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vix_change_5d = vix_change_5d.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia
vix_trend = vix_trend.shift(2).reindex(y.index)  # ✅ shift(2) por consistencia

print("✅ Features alineadas al target")

# ── CONSTRUIR SECUENCIAS TEMPORALES ─────────────────────────────
X_advanced = []
y_out = []
dates = []

print("🔧 Construyendo secuencias temporales avanzadas...")

for i in range(cfg.WINDOW, len(y)):
    # Extraer ventanas de 60 días para cada grupo de features
    ventana_ret_1d = ret_1d.iloc[i - cfg.WINDOW:i].values     # (60, 40)
    ventana_ret_3d = ret_3d.iloc[i - cfg.WINDOW:i].values     # (60, 40)
    ventana_ret_5d = ret_5d_feat.iloc[i - cfg.WINDOW:i].values # (60, 40)
    ventana_ret_10d = ret_10d.iloc[i - cfg.WINDOW:i].values   # (60, 40)
    
    ventana_vol_5d = vol_5d.iloc[i - cfg.WINDOW:i].values     # (60, 40)
    ventana_vol_20d = vol_20d.iloc[i - cfg.WINDOW:i].values   # (60, 40)
    ventana_vol_60d = vol_60d.iloc[i - cfg.WINDOW:i].values   # (60, 40)
    
    ventana_mom_3_20 = mom_3_20.iloc[i - cfg.WINDOW:i].values # (60, 40)
    ventana_mom_5_60 = mom_5_60.iloc[i - cfg.WINDOW:i].values # (60, 40)
    
    ventana_ratio_3_10 = ret_ratio_3_10.iloc[i - cfg.WINDOW:i].values # (60, 40)
    ventana_vol_ratio = vol_ratio_5_20.iloc[i - cfg.WINDOW:i].values   # (60, 40)
    ventana_market_corr = market_corr.iloc[i - cfg.WINDOW:i].values    # (60, 40)
    
    # VIX features (4 features × 1 = 4)
    ventana_vix_level = vix_level.iloc[i - cfg.WINDOW:i].values.reshape(-1, 1)     # (60, 1)
    ventana_vix_chg_1d = vix_change_1d.iloc[i - cfg.WINDOW:i].values.reshape(-1, 1) # (60, 1)
    ventana_vix_chg_5d = vix_change_5d.iloc[i - cfg.WINDOW:i].values.reshape(-1, 1) # (60, 1)
    ventana_vix_trend = vix_trend.iloc[i - cfg.WINDOW:i].values.reshape(-1, 1)     # (60, 1)
    
    # Concatenar todas las features: 484 total
    bloque = np.concatenate([
        # Retornos (160)
        ventana_ret_1d, ventana_ret_3d, ventana_ret_5d, ventana_ret_10d,
        # Volatilidades (120)
        ventana_vol_5d, ventana_vol_20d, ventana_vol_60d,
        # Momentum (80)
        ventana_mom_3_20, ventana_mom_5_60,
        # Ratios técnicos (80)
        ventana_ratio_3_10, ventana_vol_ratio,
        # Cross-asset (40)
        ventana_market_corr,
        # VIX features (4)
        ventana_vix_level, ventana_vix_chg_1d, ventana_vix_chg_5d, ventana_vix_trend
    ], axis=1)  # (60, 484)

    X_advanced.append(bloque)
    y_out.append(y.iloc[i])
    dates.append(y.index[i])

X = np.array(X_advanced, dtype=np.float32)
y = np.array(y_out, dtype=np.float32)
dates = pd.to_datetime(dates)

print(f"🚀 LSTM-5d AVANZADO:")
print(f"   ✅ X shape: {X.shape} (vs original: (4455, 60, 81))")
print(f"   ✅ y shape: {y.shape}")
print(f"   📊 Features: {X.shape[2]} (incremento {X.shape[2]/81:.1f}x)")
print(f"   🎯 Target: retornos 5-días (mantener división /5 en backtest)")


🔧 Alineando features avanzadas al target...
✅ Features alineadas al target
🔧 Construyendo secuencias temporales avanzadas...
🚀 LSTM-5d AVANZADO:
   ✅ X shape: (4458, 60, 484) (vs original: (4455, 60, 81))
   ✅ y shape: (4458, 40)
   📊 Features: 484 (incremento 6.0x)
   🎯 Target: retornos 5-días (mantener división /5 en backtest)


In [16]:
# 🔧 CREAR VENTANAS DE DATOS CON TARGET DIARIO (FIXED)
print("\n🔧 Creando ventanas de datos con target diario...")

X = []
y_out = []
dates = []

for i in range(cfg.WINDOW, min(len(ret), len(y))):
    ventana_ret = ret.iloc[i - cfg.WINDOW:i].values
    ventana_mom = momentum.iloc[i - cfg.WINDOW:i].values
    ventana_vix = vix_t1.iloc[i - cfg.WINDOW:i].values.reshape(-1, 1)
    bloque = np.concatenate([ventana_ret, ventana_mom, ventana_vix], axis=1)

    X.append(bloque)
    y_out.append(y[i])           # ← FIX: y es numpy array, usar [i] no .iloc[i]
    dates.append(ret.index[i])   # ← FIX: usar ret.index[i] no y.index[i]

X = np.array(X, dtype=np.float32)
y_final = np.array(y_out, dtype=np.float32)
dates = pd.to_datetime(dates)

print(f"✅ X shape: {X.shape}")
print(f"✅ y shape: {y_final.shape}")
print(f"✅ Features por timestep: {X.shape[2]} [retornos(40) + momentum(40) + VIX(1)]")
print(f"✅ Ventanas de datos: {X.shape[0]} muestras")
print(f"✅ Período: {dates.min().date()} → {dates.max().date()}")

# Actualizar y para ser consistente
y = y_final



🔧 Creando ventanas de datos con target diario...
✅ X shape: (4398, 60, 81)
✅ y shape: (4398, 40)
✅ Features por timestep: 81 [retornos(40) + momentum(40) + VIX(1)]
✅ Ventanas de datos: 4398 muestras
✅ Período: 2012-08-15 → 2025-04-25


In [17]:
# Ver una muestra
pd.DataFrame(X[0]).iloc[:5]  # primeros 5 días de la primera muestra

# Ver un target
pd.Series(y[0], index=df.columns).sort_values(ascending=False).head()


Ticker
META    0.043795
TSLA    0.024435
MA      0.011427
T       0.009792
DIS     0.008253
dtype: float32

In [18]:
# ─────────────────────────────────────────────
# 4. ESCALADO (features y target)
# ─────────────────────────────────────────────
from pathlib import Path      #  ← AÑADE ESTA LÍNEA

n_feat = X.shape[2]

# ---- escalador de X ----
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X.reshape(-1, n_feat)).reshape(X.shape)

# ---- escalador de y ----
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)     # y.shape = (n_muestras, n_assets)

# ---- guarda los escaladores ----
Path(cfg.MODELS).mkdir(parents=True, exist_ok=True)
joblib.dump(scaler_X, cfg.MODELS / "scaler_X_lstm5d.pkl")
joblib.dump(scaler_y, cfg.MODELS / "scaler_y_lstm5d.pkl")
print("✅ Escaladores guardados (lstm5d con VIX integrado)")


✅ Escaladores guardados (lstm5d con VIX integrado)


In [19]:
joblib.dump(
    {
        "X": X_scaled.astype(np.float32),
        "y": y_scaled.astype(np.float32),
        "tickers": df.columns.tolist(),
        "dates": dates,
        "target_type": "daily",  # ← NUEVO: indicar que es target diario
        "features": "ret_momentum_vix"  # ← NUEVO: indicar features incluidas
    },
    cfg.DATA / "processed" / "lstm5d_data.pkl"
)
print("✅ Datos LSTM-5d MEJORADOS guardados (target diario + features completas)")


✅ Datos LSTM-5d MEJORADOS guardados (target diario + features completas)
